# Notebook para realizar a limpeza do grafo após seu povoamento automático

In [1]:
from langchain_community.graphs import Neo4jGraph
from dotenv import load_dotenv
import os
import pandas as pd
import matplotlib.pyplot as plt

Conectando ao NEO4J

In [2]:
load_dotenv()

# Neo4j variables
NEO4J_URL = os.getenv("NEO4J_URL")
NEO4J_USERNAME =os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")

#Connecting to the graph
graph = Neo4jGraph(
    url=NEO4J_URL,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD
)

Ajustando as URIs das classes CNPQ

In [3]:
# Relacoes "de" e "para" dos nós CNPQ
result = graph.query(f"""
    MATCH (n:CNPQ)-[rel]->(o)
    return distinct(type(rel)) 
            """)

print (result)

result = graph.query(f"""
    MATCH (o)-[rel]->(n:CNPQ)
    return distinct(type(rel)) 
            """)

print (result)

[{'(type(rel))': 'BFO_0000051'}]
[{'(type(rel))': 'subject'}, {'(type(rel))': 'BFO_0000051'}, {'(type(rel))': 'mentions'}]


In [7]:
# Ajustando as URI. Dada uma URI incorreta, mover todas as relações para a URI correta e deletar o nó com a URI incorreta.

# URI incorreta, URI correta, novo label
lista_correcao = [
    #('tag:stardog:api:health_sciences', 'tag:stardog:api:health_science', 'CIÊNCIAS DA SAÚDE'),
    #('tag:stardog:api:applied_social_sciences', 'tag:stardog:api:applied_social_science', 'CIÊNCIA SOCIAL APLICADA'),
    #('tag:stardog:api:6090000__8_communication', 'tag:stardog:api:communication', 'COMUNICAÇÃO'),
    #('tag:stardog:api:7030000__3_anthropology', 'tag:stardog:api:anthropology	', 'ANTROPOLOGIA'),
    #('tag:stardog:api:9180000__5_social_studies', 'tag:stardog:api:social_study', 'ESTUDOS SOCIAIS'),
    #('tag:stardog:api:8020800__2_modern_foreign_literature', 'tag:stardog:api:modern_foreign_literature', 'LITERATURA ESTRANGEIRA MODERNA'),
    #('tag:stardog:api:exact_and_earth_sciences', 'tag:stardog:api:exact_and_earth_science', 'CIÊNCIAS EXATAS E DA TERRA'),
    #('tag:stardog:api:botanica','tag:stardog:api:botany', 'BOTÂNICA'),
    #('tag:stardog:api:olympiads', 'tag:stardog:api:mathematic', 'MATEMÁTICA'),
    #('tag:stardog:api:physical', 'tag:stardog:api:physic', 'FÍSICA'),
    #('tag:stardog:api:chemical', 'tag:stardog:api:chemistry', 'QUÍMICA'), 
    #('tag:stardog:api:story', 'tag:stardog:api:history', 'Hístória'),
    #('tag:stardog:api:theology_', 'tag:stardog:api:theology', 'TEOLOGIA'),
    #('tag:stardog:api:right_', 'tag:stardog:api:right', 'DIREITO'),
    ('tag:stardog:api:lyric', 'tag:stardog:api:letters', 'LETRAS')
    ]

for corr in lista_correcao:

    graph.query("""
    MERGE (m:CNPQ{uri:'""" + corr[1] + """'})
    """)
        
    graph.query("""
    MATCH (n:CNPQ{uri:'""" + corr[0] + """'})-[rel:BFO_0000051]->(o)
    MATCH (m:CNPQ{uri:'""" + corr[1] + """'})
    MERGE (m)-[:BFO_0000051]->(o)
    """)

    graph.query("""
    MATCH (s)-[:subject]->(n:CNPQ{uri:'""" + corr[0] + """'})
    MATCH (m:CNPQ{uri:'""" + corr[1] + """'})
    MERGE (s)-[:subject]->(m)
    """)

    graph.query("""
    MATCH (s)-[:BFO_0000051]->(n:CNPQ{uri:'""" + corr[0] + """'})
    MATCH (m:CNPQ{uri:'""" + corr[1] + """'})
    MERGE (s)-[:BFO_0000051]->(m)
    """)

    graph.query("""
    MATCH (s)-[:mentions]->(n:CNPQ{uri:'""" + corr[0] + """'})
    MATCH (m:CNPQ{uri:'""" + corr[1] + """'})
    MERGE (s)-[:mentions]->(m)
    """)

    graph.query("""
    MATCH (m:CNPQ{uri:'""" + corr[1] + """'})
    SET m.label = '""" + corr[2] + """'
    """)

    graph.query("""
    MATCH (m:CNPQ{uri:'""" + corr[0] + """'})
    DETACH DELETE (m)
    """)

    print (corr)

('tag:stardog:api:lyric', 'tag:stardog:api:letters', 'LETRAS')


In [4]:
# Ajustando labels das URI. Dada uma URI ajustar a label.

# URI correta, novo label
lista_correcao = [
    ('tag:stardog:api:agricultural_science', 'CIÊNCIAS AGRÁRIAS'),
    ('tag:stardog:api:biological_science', 'CIÊNCIAS BIOLÓGICAS'),
    ('tag:stardog:api:health_science', 'CIÊNCIAS DA SAÚDE'),
    ('tag:stardog:api:exact_and_earth_science', 'CIÊNCIAS EXATAS E DA TERRA'),
    ('tag:stardog:api:engineering', 'ENGENHARIAS'),
    ('tag:stardog:api:humanity', 'CIÊNCIAS HUMANAS'),
    ('tag:stardog:api:applied_social_science', 'CIÊNCIAS SOCIAIS APLICADAS'),
    ('tag:stardog:api:linguistic__letter_and_art', 'LINGÜÍSTICA, LETRAS E ARTES'),
    ('tag:stardog:api:history', 'HISTÓRIA') ]

for corr in lista_correcao:
    graph.query("""
    MATCH (m:CNPQ{uri:'""" + corr[0] + """'})
    SET m.label = '""" + corr[1] + """'
    """)

    print (corr)

('tag:stardog:api:agricultural_science', 'CIÊNCIAS AGRÁRIAS')
('tag:stardog:api:biological_science', 'CIÊNCIAS BIOLÓGICAS')
('tag:stardog:api:health_science', 'CIÊNCIAS DA SAÚDE')
('tag:stardog:api:exact_and_earth_science', 'CIÊNCIAS EXATAS E DA TERRA')
('tag:stardog:api:engineering', 'ENGENHARIAS')
('tag:stardog:api:humanity', 'CIÊNCIAS HUMANAS')
('tag:stardog:api:applied_social_science', 'CIÊNCIAS SOCIAIS APLICADAS')
('tag:stardog:api:linguistic__letter_and_art', 'LINGÜÍSTICA, LETRAS E ARTES')
('tag:stardog:api:history', 'HISTÓRIA')


In [7]:
# Deletando um nó CNPQ específico que não deveria existir
corr = ['tag:stardog:api:600',
        'tag:stardog:api:8765449414823306929',
        'tag:stardog:api:_600',
        'tag:stardog:api:_7277407233034425144_',
        'tag:stardog:api:_2134759983998029892_'
        ]

for uri in corr:
    graph.query(f"""
    MATCH (n:CNPQ{{uri:'{uri}'}})
    DETACH DELETE (n)
    """)

    print (uri)

tag:stardog:api:600
tag:stardog:api:8765449414823306929
tag:stardog:api:_600
tag:stardog:api:_7277407233034425144_
tag:stardog:api:_2134759983998029892_


In [ ]:
# Identificando teses ligadas a uma área do CNPQ
uri = 'tag:stardog:api:geoscience'

result = graph.query("""
        MATCH (n0:CNPQ{uri:'""" + uri + """'})-[:BFO_0000051]->(n1:CNPQ)-[:BFO_0000051]->(n2:CNPQ)-[:BFO_0000051]->(n3:CNPQ)--(t:Thesis) 
        Return t.uri
        UNION
        MATCH (n0:CNPQ{uri:'""" + uri + """'})-[:BFO_0000051]->(n1:CNPQ)-[:BFO_0000051]->(n2:CNPQ)--(t:Thesis) 
        Return t.uri
        UNION
        MATCH (n0:CNPQ{uri:'""" + uri + """'})-[:BFO_0000051]->(n1:CNPQ)--(t:Thesis) 
        Return t.uri
        UNION
        MATCH (n0:CNPQ{uri:'""" + uri + """'})--(t:Thesis) 
        Return t.uri
        """)


result[40:]

[{'t.uri': 'tag:stardog:api:analise_das_alteracoes_ambientais_na_lagoa_negra_viamaors__enfoque_na_qualidade_da_agua'},
 {'t.uri': 'tag:stardog:api:relacao_entre_a_concentracao_de_gelo_marinho_antartico_e_a_temperatura_minima_na_america_do_sul'},
 {'t.uri': 'tag:stardog:api:analise_de_aspectos_climatico_socioeconomico_e_ambiental_e_seus_efeitos_na_bacia_hidrografica_do_rio_urucui_preto_e_entorno'},
 {'t.uri': 'tag:stardog:api:impactos_das_mudancas_climaticas_na_biomassa_florestal_amazonica_previsao_de_perda_e_estrategias_de_conservacao_prioritarias_para_o_potencial_de_biomassa_sob_as_mudancas_climaticas'},
 {'t.uri': 'tag:stardog:api:racialidades_e_a_producao_de_artigos_cientificos_no_conhecimento_geografico_brasileiro_entre_2001_e_2018'},
 {'t.uri': 'tag:stardog:api:geoambientes_geoarqueologia_e_cenarios_de_mudancas_climaticas_na_ilha_de_marajo_amazonia_brasileira'},
 {'t.uri': 'tag:stardog:api:analises_geometrica_e_cinematica_mesomicroscopica_das_zonas_de_cisalhamento_palmeira_dos_ind

In [51]:
# Teses de Engenharia e Ciências Exatas e da Terra que foram consideradas fora do tema

teses_excluir = [
'tag:stardog:api:redes_neurais_artificiais_como_ferramenta_de_auxilio_na_operacao_e_controle_da_estacao_de_tratamento_de_agua_de_palmeira_dos_indios',
'tag:stardog:api:proposta_de_uma_metodologia_para_suporte_a_decisao_deinvestimentos_em_portfolio_de_projetos_de_pesquisa_geologica_abordagem_atraves_do_modelo_de_opcoes_de_precos_de_black_escholes_e_simulacao_de_monte_carlo',
'tag:stardog:api:desenvolvimento_e_aplicacao_de_biocoagulante_magnetico_fe3o4mo_para_remocao_do_corante_reativo_preto_5_em_meio_aquoso_sintetico_e_efluente_real_textil',
'tag:stardog:api:producao_de_biogas_em_resposta_ao_pretratamento_quimico_e_reuso_do_liquor_negro_aplicado_aos_residuos_de_aparas_de_grama',
'tag:stardog:api:modelo_matematico_para_manejo_de_black_bass_m_salmoides_lacepede_1802_em_reservatorios',
'tag:stardog:api:determinacao_de_mp25_e_black_carbon_em_decorrencia_da_queima_ilegal_de_residuos_solidos_urbanos',
'tag:stardog:api:ozonizacao_e_oxidacao_com_persulfato_de_sodio_do_corante_azo_reactive_black_5_cinetica_formacao_de_subprodutos_e_metodologia_de_superficie_de_resposta',
'tag:stardog:api:monitoramento_da_exposicao_pessoal_ao_poluente_atmosferico_black_carbon',
'tag:stardog:api:localizacao_e_planejamento_de_caminhos_para_um_robo_humanoide_e_um_robo_escravo_com_rodas',
'tag:stardog:api:uma_metodologia_para_simulacao_numerica_tridimensional_de_reservatorios_de_petroleo_utilizando_modelo_blackoil_e_formulacao_em_fracoes_massicas',
'tag:stardog:api:modelagem_computacional_e_analise_termodinamica_de_sistemas_de_geracao_de_potencia_utilizando_gaseificacao_de_licor_negro',
'tag:stardog:api:obtencao_e_caracterizacao_de_nanolubrificantes_utilizados_em_refrigeracao_aditivados_com_nanoparticulas_de_indio_in',
'tag:stardog:api:fabricacao_de_compositos_de_poli_fluoreto_de_vinilidenonegro_de_fumo_condutor_pvdfnfc_por_sinterizacao_seletiva_a_laser',
'tag:stardog:api:propriedades_opticas_de_vidros_de_fluoretos_de_indio_e_zirconio_puros_e_dopados_com_nd_ou_pr',
'tag:stardog:api:controle_mestreescravo_sem_comunicacao_para_conversores_de_potencia_aplicados_a_microrredes',
'tag:stardog:api:comparacao_entre_controle_por_inclinacao_e_controle_mestreescravo_sem_comunicacao_em_microrredes_operando_no_modo_autonomo',
'tag:stardog:api:otimizacao_multiobjetivo_de_black_start_em_microrredes_com_insercao_de_geracao_distribuida_intermitente',
'tag:stardog:api:obtencao_de_materiais_absorvedores_de_microondas_a_base_de_compositos_condutores_de_negro_de_fumo_e_nanotubos_de_carbono_impregnados_com_poma',
'tag:stardog:api:sintese_de_hidroxido_de_indio_iii_substituido_parcialmente_por_zn2_com_propriedades_fotocataliticas_e_ag_com_propriedades_bactericidas',
'tag:stardog:api:producao_e_caracterizacao_de_compositos_de_pebdal_oriundos_de_embalagens_longa_vida_com_podeborracha_de_pneus_ou_negro_de_fumo',
'tag:stardog:api:metodologia_baseada_em_metamodelos_para_a_otimizacao_global_de_problemas_blackbox_com_restricoes_computacionalmente_onerosas',
'tag:stardog:api:influencia_da_composicao_mineral_e_fluida_sobre_as_propriedades_petrofisicas_de_folhelhos_negros_estudo_de_caso_bacia_do_araripe',
'tag:stardog:api:implementacao_do_modelo_de_ordem_reduzida_tpwl_para_simulacao_de_reservatorios_de_petroleo_na_plataforma_mrst',
'tag:stardog:api:metodologia_de_selecao_de_algoritmos_para_problemas_de_otimizacao_continua_e_com_formulacao_blackbox',
'tag:stardog:api:construcao_de_tabelas_de_seno_nas_civilizacoes_grega_arabe_e_indiana',
'tag:stardog:api:teste_funcional_sistematico_estendido_uma_contribuicao_na_aplicacao_de_criterios_de_teste_caixapreta',
'tag:stardog:api:paralelizacao_hibrida_do_algoritmo_black_hole_em_um_system_on_chip',
'tag:stardog:api:proposta_de_uma_linguagem_especifica_de_dominio_de_programacao_paralela_orientada_a_padroes_paralelos_um_estudo_de_caso_baseado_no_padrao_mestreescravo_para_arquiteturas_multicore',
'tag:stardog:api:um_protocolo_de_roteamento_resistente_a_ataques_blackhole_sem_deteccao_de_nos_maliciosos',
'tag:stardog:api:an_analytic_study_of_lensing_by_black_holes_in_kerrde_sitter_spacetimes',
'tag:stardog:api:secao_de_choque_de_absorcao_de_buracos_negros_de_schwarzschild_e_de_buracos_acusticos_canonicos',
'tag:stardog:api:termodinamica_de_buracos_negros_de_schwarzschild',
'tag:stardog:api:magnetismo_d0_em_oxidos_de_in2o3_e_sn1xcexo2',
'tag:stardog:api:estudos_por_espectroscopia_raman_da_heteroestrutura_semicondutora_inxga1xpgaas',
'tag:stardog:api:contraste_entre_propriedades_optoeletronicas_em_estruturas_de_tunelamento_ressonante_nin_baseadas_em_gaas',
'tag:stardog:api:condicoes_de_energia_de_hawking_e_ellis_e_a_equacao_de_raychaudhuri',
'tag:stardog:api:radiacao_de_corpo_negro_do_campo_escalar_no_universo_de_einstein',
'tag:stardog:api:proposta_de_unidade_de_ensino_potencialmente_significativa_em_radiacao_do_corpo_negro_e_efeito_fotoeletrico',
'tag:stardog:api:optical_analogue_models_for_black_holes_and_wormholes',
'tag:stardog:api:construcao_e_avaliacao_de_eletrodos_por_serigrafia_empregando_negro_de_fumo_como_material_condutor',
'tag:stardog:api:mangue_preto_avicennia_schaueriana_na_fitorremediacao_de_hidrocarbonetos_policiclicos_aromaticos_em_sedimento_de_manguezal_contaminado_por_petroleo',
'tag:stardog:api:winds_and_feedback_from_underfed_supermassive_black_holes_ventos_e_feedback_de_buracos_negros_acretando_a_baixas_taxas',
'tag:stardog:api:paralelizacao_hibrida_do_algoritmo_black_hole_em_um_system_on_chip',
'tag:stardog:api:buraco_negros_correspondencia_adsbcft_e_fluidogravidade',
'tag:stardog:api:estudo_do_processo_de_acidificacao_de_rochas_reservatorio_por_meio_de_rmn_e_microtomografia_de_raiosx',
'tag:stardog:api:desenvolvimento_de_sistemas_microemulsionados_retardados_para_aplicacao_na_estimulacao_acida_em_reservatorios_carbonaticos']

In [56]:
# Deletando teses

for uri in teses_excluir:
    graph.query(f"""
    MATCH (t:Thesis{{uri:'{uri}'}})
    DETACH DELETE (t)
    """)

    print (uri)

# Deletando as Excerpts sem teses 
graph.query(f"""
MATCH (e:Excerpt)
WHERE NOT (e)--(:Thesis)
DETACH DELETE (e)
    """)

tag:stardog:api:redes_neurais_artificiais_como_ferramenta_de_auxilio_na_operacao_e_controle_da_estacao_de_tratamento_de_agua_de_palmeira_dos_indios
tag:stardog:api:proposta_de_uma_metodologia_para_suporte_a_decisao_deinvestimentos_em_portfolio_de_projetos_de_pesquisa_geologica_abordagem_atraves_do_modelo_de_opcoes_de_precos_de_black_escholes_e_simulacao_de_monte_carlo
tag:stardog:api:desenvolvimento_e_aplicacao_de_biocoagulante_magnetico_fe3o4mo_para_remocao_do_corante_reativo_preto_5_em_meio_aquoso_sintetico_e_efluente_real_textil
tag:stardog:api:producao_de_biogas_em_resposta_ao_pretratamento_quimico_e_reuso_do_liquor_negro_aplicado_aos_residuos_de_aparas_de_grama
tag:stardog:api:modelo_matematico_para_manejo_de_black_bass_m_salmoides_lacepede_1802_em_reservatorios
tag:stardog:api:determinacao_de_mp25_e_black_carbon_em_decorrencia_da_queima_ilegal_de_residuos_solidos_urbanos
tag:stardog:api:ozonizacao_e_oxidacao_com_persulfato_de_sodio_do_corante_azo_reactive_black_5_cinetica_formaca

[]

Corrigindo as entidades

In [ ]:
# Uma entidade foi classificada erroneamente como pessoa, mas é um local.
# A label 'person' deve ser removida, mas mantida a label 'place'

uris = [
    #'tag:stardog:api:es',
    #'tag:stardog:api:amazonia',
    #'tag:stardog:api:terra',
    #'tag:stardog:api:porto_alegre',
    #'tag:stardog:api:recife',
    #'tag:stardog:api:maranhao',
    #'tag:stardog:api:rj',
    #'tag:stardog:api:franca',
    #'tag:stardog:api:parana',
    #'tag:stardog:api:mato_grosso',
    #'tag:stardog:api:ba',
    #'tag:stardog:api:angola',
    #'tag:stardog:api:pr',
    #'tag:stardog:api:eua',
    #'tag:stardog:api:rio',
    #'tag:stardog:api:norte',
    #'tag:stardog:api:rio_negro',
    #'tag:stardog:api:goias',
    #'tag:stardog:api:ceara',
    #'tag:stardog:api:pe',
    #'tag:stardog:api:pais',
    #'tag:stardog:api:ma',
    #'tag:stardog:api:espirito_santo',
    #'tag:stardog:api:roraima',
    #'tag:stardog:api:provincia',
    #'tag:stardog:api:holanda',
    #'tag:stardog:api:equador',
]

for uri in uris:
    result = graph.query(f"""
                MATCH (n{{uri:'{uri}'}})
                REMOVE n:Person
                RETURN n
                """)
    
    print (result[0]['n']['uri'])

tag:stardog:api:equador


In [ ]:
# Uma entidade foi classificada erroneamente como local, mas é uma pessoa.
# A label 'place' deve ser removida, mas mantida a label 'person'

uris = [
    #'tag:stardog:api:oliveira',
    #'tag:stardog:api:deus',
    #'tag:stardog:api:lima',
    #'tag:stardog:api:pereira',
    #'tag:stardog:api:santos',  
    #'tag:stardog:api:ribeiro',
    #'tag:stardog:api:kaingang',
    #'tag:stardog:api:ferreira',
    #'tag:stardog:api:souza',
    #'tag:stardog:api:castro',
    #'tag:stardog:api:rodrigues',  	
]

for uri in uris:
    result = graph.query(f"""
                MATCH (n{{uri:'{uri}'}})
                REMOVE n:Place
                RETURN n
                """)
    
    print (result[0]['n']['uri'])

tag:stardog:api:rodrigues


In [37]:
# Uma entidade foi classificada erroneamente como local (ou pessoa) e uma terceira label.
# A label 'place' e 'place' deve ser removida, mas mantida as outras labels

uris = [
    #'tag:stardog:api:paz',
    #'tag:stardog:api:usp',
    #'tag:stardog:api:indigena',
    #'tag:stardog:api:negro',
    #'tag:stardog:api:sus', 
    #'tag:stardog:api:mec'   	
]

for uri in uris:
    result = graph.query(f"""
                MATCH (n{{uri:'{uri}'}})
                REMOVE n:Place:Person
                RETURN n
                """)
    
    print (result[0]['n']['uri'])

tag:stardog:api:mec


In [ ]:
# Uma entidade foi classificada erroneamente como pessoa e não deveria ser uma entidade (não é um local)
# O nó deve ser deletado

uris = [
    #'tag:stardog:api:idem',
    #'tag:stardog:api:tese',
    #'tag:stardog:api:cf',
   # 'tag:stardog:api:ibid',
   # 'tag:stardog:api:imperio',
   # 'tag:stardog:api:ed',
   # 'tag:stardog:api:prof',
   # 'tag:stardog:api:dissertacao',
   # 'tag:stardog:api:historia',
   # 'tag:stardog:api:dra',
   # 'tag:stardog:api:agradeco',
   # 'tag:stardog:api:mestrado',
   # 'tag:stardog:api:educacao',
   # 'tag:stardog:api:ibge',
   # 'tag:stardog:api:org',
   # 'tag:stardog:api:universidade_de_sao_paulo',
   # 'tag:stardog:api:ciencias_sociais',
   # 'tag:stardog:api:ess',
   # 'tag:stardog:api:governo',
   # 'tag:stardog:api:hucitec',
   # 'tag:stardog:api:op',
   # 'tag:stardog:api:saude',
   # 'tag:stardog:api:mapa',
   # 'tag:stardog:api:cultura',
   # 'tag:stardog:api:escola'	,
   #'tag:stardog:api:republica',
   ,	
]

for uri in uris:

    result = graph.query(f"""
                MATCH (n{{uri:'{uri}'}})
                RETURN labels (n) as labels
                """)
    
    labels_bool = True
    for res in result:
        labels_bool = labels_bool & (set(res['labels']) == set(['Resource', 'Person', 'Place']))
    
    # Entidade com label 'Resource', 'Person', 'Place'
    if labels_bool:

        graph.query(f"""
                    MATCH (n{{uri:'{uri}'}})
                    DETACH DELETE (n)
                    """)
    
        print (uri, ' - Entidade com label Resource, Person, Place')

    else:
        # Entidade com label 'Resource', 'Person', 'Place' mas em dois nós diferentes
        if ((result == [{'labels': ['Place']}, {'labels': ['Resource', 'Person']}]) |
           (result == [{'labels': ['Resource', 'Person']}, {'labels': ['Place']}]) |
           (result == [{'labels': ['Person']}, {'labels': ['Place']}]) |
           (result == [{'labels': ['Resource', 'Place']}]) |
           (result == [{'labels': ['Resource', 'Place']}, {'labels': ['Person']}])):
            graph.query(f"""
                    MATCH (n{{uri:'{uri}'}})
                    DETACH DELETE (n)
                    """)
            print (uri, ' - Entidade com label Resource, Person, Place mas em dois nós diferentes')
        
        # Entidade com outras labels
        else:
            print (uri, ' - Entidade com outras labels - ', result)

tag:stardog:api:mec  - Entidade com outras labels -  [{'labels': ['Resource', 'Place', 'Subject']}, {'labels': ['Person']}]


In [ ]:
# Uma uri que nao deveria ser um nó (local, pessoa nem outra classe)
# O nó deve ser deletado

uris = [
    #'tag:stardog:api:t',
    #'tag:stardog:api:j',
    #'tag:stardog:api:d',
    #'tag:stardog:api:f'
    ]

for uri in uris:

    result = graph.query(f"""
                MATCH (n{{uri:'{uri}'}})
                DETACH DELETE (n)
                """)
    print (uri)
    

tag:stardog:api:f
